In [1]:
# import libraries
import pandas as pd
import numpy as np
import json
from IPython.display import JSON
import nltk
import re
import string
from tqdm import tqdm
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from langdetect import detect

from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from scipy.sparse import save_npz, load_npz

# import own functions
import preprocessing

### Parse json files containing questions from consumers and experts

In [2]:
filepath = 'data/EPIC-QA/consumer_questions_primary.json'
consumer_df = preprocessing.parse_questions(filepath)
consumer_df.head()

,question_id,question,query,background
0,CQ101,If the common cold is a type of coronavirus an...,coronavirus vaccine landscape,Looking for information on global efforts to s...
1,CQ102,Does adequate sleep prevent COVID-19?,Sleep prevent covid,Looking for any confirmation that better sleep...
2,CQ103,Can I go home when I have COVID-19 and pneumonia?,COVID-19 hospital discharge,looking for information on leaving hospital af...
3,CQ104,What endocrine complications are linked to CO...,endocrine complications covid-19,Looking for information on damage to endocrine...
4,CQ105,How long do COVID-19 antibodies stay in your s...,COVID-19 immunity duration,Looking for information on how long a person w...


In [ ]:
# consumer_df.to_csv('data/processed/questions_consumer.csv', sep='\t')

In [ ]:
# parse json files containing questions from consumers and experts
filepath = 'data/EPIC-QA/expert_questions_primary.json'
expert_df = preprocessing.parse_questions(filepath)
expert_df.head()

In [ ]:
# expert_df.to_csv('data/processed/questions_expert.csv', sep='\t')

### Load and clean metadata file

In [2]:
metadata = preprocessing.clean_metadata('data/cord-19/metadata.csv')

CSV file loaded successfully
Dropping duplicate uids


/mnt/c/Users/omlea/Documents/_training/Lighthouse/lh-final-project/covid-lit-search/preprocessing.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicates_df['nans'] = duplicates_df.apply(lambda x: x.isnull().sum(), axis=1)
  0%|          | 11/24167 [00:00<03:54, 103.22it/s]

Choosing rows to drop


100%|██████████| 24167/24167 [04:27<00:00, 90.20it/s] 


Dropping duplicate sha ids


 50%|█████     | 9/18 [00:00<00:00, 85.06it/s]

Choosing rows to drop


100%|██████████| 18/18 [00:00<00:00, 74.38it/s]


Metadata cleaning complete


In [3]:
# metadata.to_csv('data/processed/metadata_clean.csv', sep='\t')

### Create cleaned title and abstract strings

In [4]:
metadata = preprocessing.load_cleaned_metadata('data/processed/metadata_clean.csv')

In [5]:
metadata['search_text'] = preprocessing.make_search_documents(metadata)

100%|██████████| 508052/508052 [39:31<00:00, 214.22it/s] 


In [14]:
metadata['search_text_length'] = metadata.search_text.apply(lambda x: len(x.split(' ')))

In [16]:
# remove rows where search_text is less than six words
metadata = metadata[metadata.search_text_length > 5]

In [19]:
# metadata.to_csv('data/processed/02_full_table.csv', sep='\t')

In [18]:
# metadata['language'] = [detect(x) for x in tqdm(metadata.search_text.values)]

  1%|          | 2938/472361 [01:07<2:30:05, 52.13it/s]

In [16]:
# # remove non-english (redundant)

# df = pd.read_csv('data/processed/02_full_table.csv', index_col=0)

# langs = pd.read_csv('data/processed/languages.csv', index_col=0)['0']
# df['language'] = langs
# df = df[df.language == 'en'].drop(columns='language')

# # remove remaining null rows
# df.dropna(inplace=True)